In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from bayesian_testing.metrics import eval_bernoulli_agg, eval_delta_lognormal_agg

In [2]:
df = pd.read_csv("data/session_data.csv")

In [3]:
# example session data - each row represent one session
len(df)
df.head()

94500

,conversion,date,revenue,source,variant
0,0,2021-08-07,0.000000,desktop,B
1,1,2021-08-05,7.241015,desktop,C
2,0,2021-08-06,0.000000,desktop,A
3,0,2021-08-05,0.000000,desktop,C
4,0,2021-08-03,0.000000,desktop,A


In [4]:
# summary statistics per variant

summary = df.groupby('variant')[['variant', 'conversion', 'revenue']]\
            .agg({'variant': 'count', 'conversion': 'sum','revenue': 'sum'})\
            .rename(columns = {'variant': 'sessions', 'conversion': 'conversions'})

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['revenue_per_session'] = summary['revenue'] / summary['sessions']
summary['revenue_per_converted_sessions'] = summary['revenue'] / summary['conversions']

summary

,sessions,conversions,revenue,conversion_rate,revenue_per_session,revenue_per_converted_sessions
variant,,,,,,
A,31500,1580,30830.025613,0.050159,0.978731,19.512674
B,32000,1700,35203.216888,0.053125,1.100101,20.707775
C,31000,1550,37259.563364,0.050000,1.201921,24.038428


In [5]:
variant_A = df['revenue'][(df.variant == 'A')].values
variant_B = df['revenue'][(df.variant == 'B')].values
variant_C = df['revenue'][(df.variant == 'C')].values

sessions = [
    variant_A.size,
    variant_B.size,
    variant_C.size
]

conversions = [
    sum(variant_A > 0),
    sum(variant_B > 0),
    sum(variant_C > 0)
]

sum_log_revenue = [
    np.log(variant_A[variant_A > 0]).sum(),
    np.log(variant_B[variant_B > 0]).sum(),
    np.log(variant_C[variant_C > 0]).sum()
]

sum_log_2_revenue = [
    np.square(np.log(variant_A[variant_A > 0])).sum(),
    np.square(np.log(variant_B[variant_B > 0])).sum(),
    np.square(np.log(variant_C[variant_C > 0])).sum()
]

sessions
conversions
sum_log_revenue
sum_log_2_revenue

[31500, 32000, 31000]

[1580, 1700, 1550]

[3831.806394737816, 4211.72986767986, 4055.965234848171]

[11029.923165846496, 12259.51868396913, 12357.911862914]

## Results

In [6]:
# conversion rate probabilities of being best, expected loss and credible intervals for each variant
pbbs, loss, intervals = eval_bernoulli_agg(sessions, conversions)
print(f"Probabilities of being best: {pbbs}")
print(f"Expected loss: {loss}")
print(f"95% credible intervals: {intervals}")

Probabilities of being best: [0.04215, 0.92285, 0.035]
Expected loss: [0.0030094, 5.86e-05, 0.0032051]
95% credible intervals: [[0.0478136, 0.0526138], [0.0506898, 0.0555755], [0.0475854, 0.0524707]]


In [7]:
# revenue per session probabilities of being best, expected loss and credible intervals for each variant
pbbs, loss, intervals = eval_delta_lognormal_agg(sessions, conversions, sum_log_revenue, sum_log_2_revenue)
print(f"Probabilities of being best: {pbbs}")
print(f"Expected loss: {loss}")
print(f"95% credible intervals: {intervals}")

Probabilities of being best: [0.00025, 0.0345, 0.96525]
Expected loss: [0.2205845, 0.1205336, 0.0008507]
95% credible intervals: [[0.9083484, 1.0669433], [1.0039529, 1.169006], [1.1092165, 1.3053917]]


### Results for "being best" = "being minimum"

In [8]:
# conversion rate probabilities of being best, expected loss and credible intervals for each variant
pbbs, loss, intervals = eval_bernoulli_agg(sessions, conversions, min_is_best=True)
print(f"Probabilities of being best: {pbbs}")
print(f"Expected loss: {loss}")
print(f"95% credible intervals: {intervals}")

Probabilities of being best: [0.45225, 0.01065, 0.5371]
Expected loss: [0.0007916, 0.0037242, 0.000628]
95% credible intervals: [[0.0477928, 0.0526182], [0.0507342, 0.0555998], [0.0475664, 0.0524791]]


In [9]:
# revenue per session probabilities of being best, expected loss and credible intervals for each variant
pbbs, loss, intervals = eval_delta_lognormal_agg(sessions, conversions, sum_log_revenue, sum_log_2_revenue, min_is_best=True)
print(f"Probabilities of being best: {pbbs}")
print(f"Expected loss: {loss}")
print(f"95% credible intervals: {intervals}")

Probabilities of being best: [0.9552, 0.04445, 0.00035]
Expected loss: [0.0010368, 0.1007368, 0.2202043]
95% credible intervals: [[0.9074834, 1.0659393], [1.0042571, 1.1693355], [1.1088342, 1.3046563]]
